**Project Description:** The user enters a verse from the Qur’an, and the system prints the location of the verse in the Qur’an (The jozz , page number, surah name, surah number, verse number, beginning and end of the line, and interpretation of the verse.). The system also predicts the next word in the verse based on the two previous words that the user enters.

In [226]:
#import Necessary libraries
import pandas as pd
import numpy as np
import random
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [227]:
#read dataset
dataset = pd.read_csv('/content/tafseerMouaser_v03.csv', encoding='utf-8')
# Get the shape of the dataset (number of rows and columns)
dataset.shape

(6236, 12)

In [228]:
# Display the first 5 rows of the dataset
dataset.head()

,id,jozz,page,sura_no,sura_name_en,sura_name_ar,line_start,line_end,aya_no,aya_text,aya_text_emlaey,aya_tafseer
0,1,1,1,1,Al-Fātiḥah,الفَاتِحة,2,2,1,بِسۡمِ ٱللَّهِ ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ ﰀ,بسم الله الرحمن الرحيم,[1] أبتدئ قراءة القرآن باسم الله مستعيناً به، ...
1,2,1,1,1,Al-Fātiḥah,الفَاتِحة,3,3,2,ٱلۡحَمۡدُ لِلَّهِ رَبِّ ٱلۡعَٰلَمِينَ ﰁ,الحمد لله رب العالمين,[2] الثناء على الله بصفاته التي كلُّها أوصاف ك...
2,3,1,1,1,Al-Fātiḥah,الفَاتِحة,4,4,3,ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ ﰂ,الرحمن الرحيم,[3] <span class='aya'>ﵡٱلرَّحۡمَٰنِﵠ</span> ذي...
3,4,1,1,1,Al-Fātiḥah,الفَاتِحة,4,4,4,مَٰلِكِ يَوۡمِ ٱلدِّينِ ﰃ,مالك يوم الدين,[4] وهو سبحانه وحده مالك يوم القيامة، وهو يوم ...
4,5,1,1,1,Al-Fātiḥah,الفَاتِحة,5,5,5,إِيَّاكَ نَعۡبُدُ وَإِيَّاكَ نَسۡتَعِينُ ﰄ,إياك نعبد وإياك نستعين,[5] إنا نخصك وحدك بالعبادة، ونستعين بك وحدك في...


In [229]:
# Display the last 5 rows of the dataset
dataset.tail()

,id,jozz,page,sura_no,sura_name_en,sura_name_ar,line_start,line_end,aya_no,aya_text,aya_text_emlaey,aya_tafseer
6231,6232,30,604,114,An-Nās,النَّاس,12,12,2,مَلِكِ ٱلنَّاسِ ﰁ,ملك الناس,[2] ملك الناس المتصرف في كل شؤونهم، الغنيِّ عنهم.
6232,6233,30,604,114,An-Nās,النَّاس,12,13,3,إِلَٰهِ ٱلنَّاسِ ﰂ,إله الناس,[3] إله الناس الذي لا معبود بحق سواه.
6233,6234,30,604,114,An-Nās,النَّاس,13,13,4,مِن شَرِّ ٱلۡوَسۡوَاسِ ٱلۡخَنَّاسِ ﰃ,من شر الوسواس الخناس,[4] من أذى الشيطان الذي يوسوس عند الغفلة، ويخت...
6234,6235,30,604,114,An-Nās,النَّاس,13,14,5,ٱلَّذِي يُوَسۡوِسُ فِي صُدُورِ ٱلنَّاسِ ﰄ,الذي يوسوس في صدور الناس,[5] الذي يبثُّ الشر والشكوك في صدور الناس.
6235,6236,30,604,114,An-Nās,النَّاس,15,15,6,مِنَ ٱلۡجِنَّةِ وَٱلنَّاسِ ﰅ,من الجنة والناس,[6] من شياطين الجن والإنس.


In [230]:
 # Get detailed information about the dataset, including data types, memory usage, and number of missing values
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6236 entries, 0 to 6235
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               6236 non-null   int64 
 1   jozz             6236 non-null   int64 
 2   page             6236 non-null   int64 
 3   sura_no          6236 non-null   int64 
 4   sura_name_en     6236 non-null   object
 5   sura_name_ar     6236 non-null   object
 6   line_start       6236 non-null   int64 
 7   line_end         6236 non-null   int64 
 8   aya_no           6236 non-null   int64 
 9   aya_text         6236 non-null   object
 10  aya_text_emlaey  6236 non-null   object
 11  aya_tafseer      6236 non-null   object
dtypes: int64(7), object(5)
memory usage: 584.8+ KB


In [231]:
#@title Preprocessing data
#clean dataset
import string , re
import functools, operator
!pip install pyarabic
import pyarabic.araby as ar


def data_cleaning (text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'^http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"https\S+", "", text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub("(\s\d+)","",text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text = re.sub("\d+", " ", text)
    text = ar.strip_tashkeel(text)
    text = ar.strip_tatweel(text)
    text = text.replace("#", " ");
    text = text.replace("@", " ");
    text = text.replace("_", " ");

    # Remove arabic signs
    text = text[0:2] + ''.join([text[i] for i in range(2, len(text)) if text[i]!=text[i-1] or text[i]!=text[i-2]])
    text =  re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', text)

    return text

In [232]:
#cleaning apply
dataset['aya_tafseer']=dataset['aya_tafseer'].apply(lambda x: data_cleaning(x))
dataset['aya_text_emlaey']=dataset['aya_text_emlaey'].apply(lambda x: data_cleaning(x))

In [233]:
# Get the column names (keys) of the dataset
dataset.keys()

Index(['id', 'jozz', 'page', 'sura_no', 'sura_name_en', 'sura_name_ar',
       'line_start', 'line_end', 'aya_no', 'aya_text', 'aya_text_emlaey',
       'aya_tafseer'],
      dtype='object')

In [234]:
#Delete useless columns
columns_to_delete = ['sura_name_en', 'aya_text']
dataset = dataset.drop(columns_to_delete, axis=1)
dataset.head()

,id,jozz,page,sura_no,sura_name_ar,line_start,line_end,aya_no,aya_text_emlaey,aya_tafseer
0,1,1,1,1,الفَاتِحة,2,2,1,بسم الله الرحمن الرحيم,أبتدئ قراءة القرآن باسم الله مستعينا به ﵡ ٱل...
1,2,1,1,1,الفَاتِحة,3,3,2,الحمد لله رب العالمين,الثناء على الله بصفاته التي كلها أوصاف كمال و...
2,3,1,1,1,الفَاتِحة,4,4,3,الرحمن الرحيم,ﵡٱلرحمنﵠ ذي الرحمة العامة الذي وسعت رحمته جم...
3,4,1,1,1,الفَاتِحة,4,4,4,مالك يوم الدين,وهو سبحانه وحده مالك يوم القيامة وهو يوم الجز...
4,5,1,1,1,الفَاتِحة,5,5,5,إياك نعبد وإياك نستعين,إنا نخصك وحدك بالعبادة ونستعين بك وحدك في جمي...


In [235]:
# Find the most frequent value in 'sura_name_ar' to find the longest surah in the Quran
column_name = 'sura_name_ar'
most_frequent_value = dataset[column_name].mode()[0]

print("the longest surah in the Quran is: ",most_frequent_value)

the longest surah in the Quran is:  البَقَرَة 


In [236]:
# Find the least frequent value in 'sura_name_ar' to find the shortest surah in the Quran
column_name = 'sura_name_ar'
least_frequent_value = dataset[column_name].value_counts().idxmin()

print("The shortest surah in the Quran is:",least_frequent_value)

The shortest surah in the Quran is: العَصر 


In [237]:
with open('/content/tafseerMouaser_v03.csv', 'r', encoding='utf-8') as f:
    # Read the contents of the file into a string
    data = f.read()

# Tokenize the data into a list of words
tokens = nltk.word_tokenize(data)

# Open a new file called tokens.txt in write mode
with open('tokens.txt', 'w', encoding='utf-8') as f:
    # Write the tokens to the tokens.txt file, one token per line
   f.write('\n'.join(tokens))

In [238]:
# Print the number of elements (tokens) in the tokens list.
print("The length of the tokens =", len(tokens))

# Print the first five tokens in the list
print("\nThe first 9 words are:\n", tokens[:9])

The length of the tokens = 483287

The first 9 words are:
 ['id', ',', 'jozz', ',', 'page', ',', 'sura_no', ',', 'sura_name_en']


In [239]:
#@title TF-IDF
#function to return information of verse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_verse_information(query, dataset):
    # Example DataFrame with 'aya_text_emlaey' and 'aya_information' columns
    dataset_copy = dataset.copy('/content/tafseerMouaser_v03.csv')

    # Add the 'Grouped' column
    dataset_copy['aya_information'] = 'aya in jozz: ' + dataset_copy['jozz'].astype(str) + '\naya in page: ' + dataset_copy['page'].astype(str) + '\naya number in Quran: ' + dataset_copy['sura_no'].astype(str) + '\nthis aya in sura: ' + dataset_copy['sura_name_ar'].astype(str) + '\nstart lin of aya: ' + dataset_copy['line_start'].astype(str) + ' - End line of aya: ' + dataset_copy['line_end'].astype(str) + '\naya number: ' + dataset_copy['aya_no'].astype(str) + '\naya Tafser: ' + dataset_copy['aya_tafseer'].astype(str)

    # Preprocess the DataFrame (if necessary)

    # Create a TF-IDF vectorizer object
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on the 'aya_text_emlaey' column and transform the documents into TF-IDF vectors
    tfidf_vectors = vectorizer.fit_transform(dataset_copy['aya_text_emlaey'])

    # Preprocess the query
    query_vector = vectorizer.transform([query])

    # Compute cosine similarity between query vector and document vectors
    similarity_scores = cosine_similarity(query_vector, tfidf_vectors)

    # Get the indices of the most similar documents (top-k documents)
    top_k = 1
    top_indices = similarity_scores.argsort()[0][-top_k:][::-1]

    # Retrieve and store the most relevant documents along with their information
    results = []
    for index in top_indices:
        document = dataset_copy['aya_text_emlaey'].iloc[index]
        information = dataset_copy['aya_information'].iloc[index]
        results.append({'Text of the verse': document, 'Information': information})

    return results

In [240]:
#@title Build a 3-gram model
# Set the value of n to 3, indicating a 3-gram model
n = 3

# Create an empty dictionary called ngrams to store 3-gram information
ngrams = {}

# Iterate through the list of tokens, starting from index 0 to len(tokens) - n
for i in range(len(tokens) - n):
    # Create a 3-gram word by joining the current token and the next n tokens
    word = ' '.join(tokens[i:i + n])

    # Check if the 3-gram already exists in the ngrams dictionary
    if word not in ngrams.keys():
        # If the 3-gram is not present, add it as a key to the ngrams dictionary with an empty list as its value
        ngrams[word] = []

    # Append the next token to the list associated with the current 3-gram
    ngrams[word].append(tokens[i + n])

In [241]:
# Print the values for the 3-gram "إياك نعبد وإياك"
print('Display the values for the 3-gram "اياك نعبد واياك":')
print(ngrams['إياك نعبد وإياك'])

Display the values for the 3-gram "اياك نعبد واياك":
['نستعين']


In [242]:
# Set the start word to  "إياك نعبد وإياك"
start_word = "إياك نعبد وإياك"
# Get the list of words that can follow the start word from the ngrams dictionary
words = ngrams[start_word]
# Choose a random word from the list of possible following words
next_word = random.choice(words)
# Print the start word and the randomly chosen next word
print(start_word, next_word)

إياك نعبد وإياك نستعين


In [243]:
def generate_aya(start_phrase):
    start = start_phrase
    threshold = random.random()
    aya = []
    while start not in ngrams.keys():
        start_new = input("النموذج لا يدعم هذا: ")
        start = start_new
    for i in range(1):
        line = start
        for j in range(3):
            line_words = nltk.word_tokenize(line)
            start = ' '.join(line_words[-n + 3:])
            candidates = ngrams.get(start, [])
            if not candidates:
                print(f"{start}")
                break
            probability = [candidates.count(word) / len(candidates) for word in candidates]
            accumulator = [sum(probability[:i + 1]) for i in range(len(probability))]
            for word, acc in zip(candidates, accumulator):
                if acc >= threshold:
                    next_word = word
                    break
            else:
                next_word = random.choice(candidates)
            line += ' ' + next_word
        aya.append(line)
        start = random.choice(ngrams.get(start, list(ngrams.keys())))
    return '\n'.join(aya)

In [244]:
start_word = input("أدخل كلمة لبدء انشاء الايه: ")  # Prompt the user to enter a word to start the aya
#print('The ayah')
aya = generate_aya(start_word)  # Generate the aya using the input word as the starting point

query =aya
search_results = retrieve_verse_information(query, dataset)
print(f"Text of the verse: {result['Document']}")
print(f"{result['Information']}")

أدخل كلمة لبدء انشاء الايه: إياك نعبد وإياك
إياك نعبد وإياك نستعين
Text of the verse: إياك نعبد وإياك نستعين
aya in jozz: 1
aya in page: 1
aya number in Quran: 1
this aya in sura: الفَاتِحة 
start lin of aya: 5 - End line of aya: 5
aya number: 5
aya Tafser:  انا نخصك وحدك بالعبادة ونستعين بك وحدك في جميع امورنا فالامر كله بيدك لا يملك منه احد مثقال ذرةوفي هذه الاية دليل على ان العبد لا يجوز له ان يصرف شييا من انواع العبادة كالدعاء والاستغاثة والذبح والطواف الا لله وحده وفيها شفاء القلوب من داء التعلق بغير الله ومن امراض الرياء والعجب والكبرياء


In [81]:
#Candidate words
start=start_word
print(ngrams.get(start, []))

['نستعين']


In [82]:
#@title DL Part
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pandas as pd
import re
import numpy as np
import string

In [83]:
# Load dataset
#dataset = pd.read_csv('/content/tafseerMouaser_v03.csv', encoding='utf-8')

In [84]:
#@title Preprocessing data
def clean_data(doc):
    # Split the document into tokens (words)
    tokens = doc.split()

    # Remove punctuation from each token
    tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens]

    # Convert tokens to lowercase
    tokens = [word.lower() for word in tokens]

    # Join the cleaned tokens back into a single string
    cleaned_doc = ' '.join(tokens)

    return cleaned_doc

In [85]:
dataset['aya_text_emlaey'] = dataset['aya_text_emlaey'].apply(clean_data)

In [86]:
dataset['aya_text_emlaey'][0]

'بسم الله الرحمن الرحيم'

In [87]:
# Tokenize the dataset
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(dataset['aya_text_emlaey'])
sequences = tokenizer.texts_to_sequences(dataset['aya_text_emlaey'])

In [88]:
# Pad the sequences to have the same length
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_sequence_length)

In [89]:
# Split the dataset into training and testing sets
train_size = int(0.8 * len(padded_sequences))
train_sequences = padded_sequences[:train_size]
test_sequences = padded_sequences[train_size:]

In [90]:
# Convert target sequences to one-hot encoded format (convert categorical data [text] to numerical represtion)
one_hot_train_sequences = tf.keras.utils.to_categorical(train_sequences, num_classes=len(tokenizer.word_index) + 1)
one_hot_test_sequences = tf.keras.utils.to_categorical(test_sequences, num_classes=len(tokenizer.word_index) + 1)

In [91]:
#model
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
dataset_copy = pd.read_csv('/content/tafseerMouaser_v03.csv')

# Prepare your dataset
ngram_data = dataset_copy['aya_text_emlaey']

# Set the value of n
n = 3

# Convert n-gram data to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(ngram_data)
sequences = tokenizer.texts_to_sequences(ngram_data)

# Split sequences into input and target sequences
input_sequences = []
target_sequences = []
for sequence in sequences:
    for i in range(n, len(sequence)):
        input_sequences.append(sequence[i-n:i])
        target_sequences.append(sequence[i])

# Convert input and target sequences to numpy arrays
X = np.array(input_sequences)
y = np.array(target_sequences)

# Pad input sequences
X_padded = pad_sequences(X)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X_padded.shape[1]))
model.add(LSTM(100))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X_padded, y, epochs=10, batch_size=32)

Epoch 1/10
1845/1845 [==============================] - 156s 83ms/step - loss: 8.0915 - accuracy: 0.0407
Epoch 2/10
1845/1845 [==============================] - 142s 77ms/step - loss: 7.4434 - accuracy: 0.0500
Epoch 3/10
1845/1845 [==============================] - 136s 74ms/step - loss: 6.8783 - accuracy: 0.0758
Epoch 4/10
1845/1845 [==============================] - 134s 73ms/step - loss: 6.1894 - accuracy: 0.1135
Epoch 5/10
1845/1845 [==============================] - 131s 71ms/step - loss: 5.4634 - accuracy: 0.1615
Epoch 6/10
1845/1845 [==============================] - 133s 72ms/step - loss: 4.7638 - accuracy: 0.2254
Epoch 7/10
1845/1845 [==============================] - 133s 72ms/step - loss: 4.1300 - accuracy: 0.2976
Epoch 8/10
1845/1845 [==============================] - 133s 72ms/step - loss: 3.5608 - accuracy: 0.3791
Epoch 9/10
1845/1845 [==============================] - 133s 72ms/step - loss: 3.0523 - accuracy: 0.4612
Epoch 10/10
1845/1845 [==============================] 

In [95]:
#generate next word
def generate(model, tokenizer, start_word, num_words=1):
    generated = [start_word]
    for _ in range(num_words):
        # Tokenize the current word
        current_sequence = tokenizer.texts_to_sequences([start_word])
        padded_current_sequence = tf.keras.preprocessing.sequence.pad_sequences(current_sequence, maxlen=model.input_shape[1])
        # Predict the next word probabilities
        probabilities = model.predict(padded_current_sequence, verbose=0)[0]
        # Choose the top words based on probability
        top_indices = np.argsort(probabilities)[-5:]  # Adjust the number of top words as needed
        # Select a word randomly from the top predicted words
        predicted_word_index = np.random.choice(top_indices.flatten())
        predicted_word = tokenizer.index_word.get(predicted_word_index, '')
        # Append the predicted word to the generated
        generated.append(predicted_word)
        # Update the start word for the next iteration
        start_word = predicted_word if predicted_word else generated[-3]
    return ' '.join(generated)

In [96]:
start_word = input("أدخل كلمة لبدء أنشاء الايه:")  # Prompt the user to enter a word to start the ayah
generated = generate(model, tokenizer, start_word, num_words=1)

query =generated
search_results = retrieve_verse_information(query, dataset)
print("Generated ayah :",generated)
print(f"Text of the verse: {result['Document']}")
print(f"{result['Information']}")

أدخل كلمة لبدء أنشاء الايه:إياك نعبد وإياك
Generated ayah : إياك نعبد وإياك قدمتموه
Text of the verse: إياك نعبد وإياك نستعين
aya in jozz: 1
aya in page: 1
aya number in Quran: 1
this aya in sura: الفَاتِحة 
start lin of aya: 5 - End line of aya: 5
aya number: 5
aya Tafser:  انا نخصك وحدك بالعبادة ونستعين بك وحدك في جميع امورنا فالامر كله بيدك لا يملك منه احد مثقال ذرةوفي هذه الاية دليل على ان العبد لا يجوز له ان يصرف شييا من انواع العبادة كالدعاء والاستغاثة والذبح والطواف الا لله وحده وفيها شفاء القلوب من داء التعلق بغير الله ومن امراض الرياء والعجب والكبرياء
